In [1]:
%config InlineBackend.figure_formats = {"retina", "png"}
# %matplotlib notebook

# import logging
# logging.basicConfig(level=logging.INFO)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 14

import superscreen as sc

from huber_squid import huber_squid, huber_geometry

In [2]:
sc.about.version_dict()

{'SuperScreen': '0.1.1',
 'Numpy': '1.21.0',
 'SciPy': '1.7.0',
 'matplotlib': '3.4.2',
 'ray': '1.4.1',
 'IPython': '7.25.0',
 'Python': '3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)]',
 'OS': 'nt [win32]',
 'Number of CPUs': 4,
 'BLAS Info': 'OPENBLAS'}

In [3]:
sc.about.version_table()

Software,Version
SuperScreen,0.1.1
Numpy,1.21.0
SciPy,1.7.0
matplotlib,3.4.2
ray,1.4.1
IPython,7.25.0
Python,"3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)]"
OS,nt [win32]
Number of CPUs,4
BLAS Info,OPENBLAS


In [ ]:
import time

t0 = time.time()

In [ ]:
def huber_field_coil():
    squid = huber_squid()
    
    return sc.Device(
        "huber_field_coil",
        layers=[squid.layers["BE"]],
        films=[squid.films["fc"]],
        holes=[squid.holes["fc_center"]],
        abstract_regions=[squid.abstract_regions["bounding_box"]],
        length_units=squid.length_units,
    )

def huber_without_field_coil():
    squid = huber_squid()
    
    layers = squid.layers
    _ = layers.pop("BE")
    films = squid.films
    _ = films.pop("fc")
    # _ = films.pop("fc_shield", None)
    holes = squid.holes
    _ = holes.pop("fc_center")
    
    return sc.Device(
        "huber_without_field_coil",
        layers=layers,
        films=films,
        holes=holes,
        abstract_regions=squid.abstract_regions,
        length_units=squid.length_units,
    )

In [ ]:
sample_points = np.array(
    [
        [-12.0, -15.0],
        [-12.0,  12.0],
        [ 15.0,  12.0],
        [ 15.0, -15.0],
    ]
)

sample_height = -1.0

min_triangles = 8000

In [ ]:
field_coil = huber_field_coil()

abstract_regions = field_coil.abstract_regions
abstract_regions["bounding_box"] = sc.Polygon(
    "bounding_box",
    layer=abstract_regions["bounding_box"].layer,
    points=sample_points,
)

field_coil.abstract_regions = abstract_regions

In [ ]:
field_coil.make_mesh(min_triangles=min_triangles, optimesh_steps=200)

In [ ]:
ax = field_coil.plot_mesh()
ax = field_coil.plot_polygons(ax=ax, color='k', lw=3, legend=False)
_ = ax.set_title(
    f"Mesh: {field_coil.points.shape[0]} points, "
    f"{field_coil.triangles.shape[0]} triangles"
)
ax.figure.set_size_inches(8,8)

In [ ]:
t0 = time.time()

for _ in range(16):
    applied_field = sc.sources.ConstantField(0.0)

    circulating_currents = {
        "fc_center": "1 mA"
    }

    solutions = sc.solve(
        device=field_coil,
        applied_field=applied_field,
        circulating_currents=circulating_currents,
        field_units="Phi_0/um**2",
        current_units="uA",
        iterations=6,
        coupled=True,
    )
    field_coil_solution = solutions[-1]
    
print(f"Elapsed time: {time.time() - t0:.3f} seconds")

In [ ]:
fig, axes = sc.plot_currents(
    field_coil_solution,
    figsize=(5,6),
    streamplot=True,
    units="mA/um",
    cross_section_xs=0,
    cross_section_angle=0,
)
for ax in axes:
    field_coil.plot_polygons(ax=ax, legend=False, color='w', lw=0.75, alpha=0.75)

In [ ]:
t0 = time.time()

circulating_currents = [{"fc_center": f"1 mA"} for _ in range(16)]

solutions, paths = sc.solve_many(
    parallel_method=None,
    device=field_coil,
    directory=None,
    return_solutions=False,
    keep_only_final_solution=True,
    applied_fields=applied_field,
    circulating_currents=circulating_currents,
    field_units="Phi_0/um**2",
    current_units="uA",
    iterations=6,
    coupled=True,
)

print(f"Elapsed time: {time.time() - t0:.3f} seconds")

In [ ]:
t0 = time.time()

circulating_currents = [{"fc_center": f"1 mA"} for _ in range(16)]

solutions, paths = sc.solve_many(
    parallel_method="mp",
    device=field_coil,
    directory=None,
    return_solutions=False,
    keep_only_final_solution=True,
    applied_fields=applied_field,
    circulating_currents=circulating_currents,
    field_units="Phi_0/um**2",
    current_units="uA",
    iterations=6,
    coupled=True,
)

print(f"Elapsed time: {time.time() - t0:.3f} seconds")

In [ ]:
import ray
ray.init()

In [ ]:
t0 = time.time()

circulating_currents = [{"fc_center": f"1 mA"} for _ in range(16)]

solutions, paths = sc.solve_many(
    parallel_method="ray",
    device=field_coil,
    directory=None,
    return_solutions=False,
    keep_only_final_solution=True,
    applied_fields=applied_field,
    circulating_currents=circulating_currents,
    field_units="Phi_0/um**2",
    current_units="uA",
    iterations=6,
    coupled=True,
)

print(f"Elapsed time: {time.time() - t0:.3f} seconds")

## Simulate mutual inductance without a sample

In [ ]:
# squid = huber_without_field_coil()

# layers = squid.layers
# # layers["sample_layer"] = sc.Layer("sample_layer", Lambda=4, z0=-0.5)

# films = squid.films
# # films["sample"] = sc.Polygon(
# #     "sample",
# #     layer="sample_layer",
# #     points=sample_points,
# # )

# abstract_regions = squid.abstract_regions
# abstract_regions["bounding_box"] = sc.Polygon(
#     "bounding_box",
#     layer=abstract_regions["bounding_box"].layer,
#     points=sample_points,
# )

# squid.layers = layers
# squid.films = films

In [ ]:
squid = huber_without_field_coil()

Lambda = sc.Constant(1000)

layers = squid.layers
layers["sample_layer"] = sc.Layer("sample_layer", Lambda=Lambda, z0=sample_height)

films = squid.films
films["sample"] = sc.Polygon(
    "sample",
    layer="sample_layer",
    points=sample_points,
)

abstract_regions = squid.abstract_regions
abstract_regions["bounding_box"] = sc.Polygon(
    "bounding_box",
    layer=abstract_regions["bounding_box"].layer,
    points=sample_points,
)

squid.layers = layers
squid.films = films
squid.abstract_regions = abstract_regions

In [ ]:
squid.make_mesh(min_triangles=min_triangles, optimesh_steps=200)

In [ ]:
ax = squid.plot_mesh()
ax = squid.plot_polygons(ax=ax, color='k', lw=3, legend=False)
_ = ax.set_title(
    f"Mesh: {squid.points.shape[0]} points, "
    f"{squid.triangles.shape[0]} triangles"
)
ax.figure.set_size_inches(8,8)

In [ ]:
field_units = "mT"

fc_fields = {}
for layer in squid.layers_list:
    fc_fields[layer.z0] = field_coil_solution.field_at_position(
        squid.points,
        zs=layer.z0,
        units=field_units,
        with_units=False,
    )

def field_coil_field(x, y, z, solution=field_coil_solution, units="mT"):
    return fc_fields[z]

In [ ]:
applied_field = field_coil_field

circulating_currents = None

solutions = sc.solve(
    device=squid,
    applied_field=applied_field,
    circulating_currents=circulating_currents,
    field_units="mT",
    current_units="uA",
    iterations=6,
    coupled=True,
)

In [ ]:
# records = []
# for s in solutions:
#     records.append(s.polygon_flux(units="Phi_0", with_units=False))
# df = pd.DataFrame.from_records(records)
# df.index.name = "Iteration"

In [ ]:
# df

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))
# ax.grid(True)
# for col in df.columns:
#     ys = df[col].values
#     ys = np.abs(np.diff(ys)[1:] / ys[1:-1])
#     xs = np.arange(len(ys)) + 1
#     ax.plot(xs, ys, 'o--', label=col)
# ax.set_ylabel("Fractional change in flux\n$(\\Phi_{(i)} -\\Phi_{(i-1)}) / \\Phi_{(i-1)}$", fontsize=14)
# ax.set_xlabel("Iteration", fontsize=14)
# ax.set_xticks(xs)
# ax.set_yscale("log")
# ax.legend(loc=0)
# fig.tight_layout()

In [ ]:
flux = solutions[-1].polygon_flux()
I_circ = squid.ureg(field_coil_solution.circulating_currents["fc_center"])
print(f"{flux['pl_hull'].to('Phi_0'):.3e~P}")
print(
    f"{(flux['pl_hull'] / I_circ).to('Phi_0/A'):.3f~P} = "
    f"{(flux['pl_hull'] / I_circ).to('pH'):.3f~P}"
)

In [ ]:
for solution in solutions[-1:]:
    fig, axes = sc.plot_fields(
        solution,
        layers=["sample_layer", "W2", "W1"],
        units="mT",
        cmap="cividis",
        figsize=(16, 6),
        grid_shape=(400, 400),
        cross_section_xs=None,
        cross_section_angle=0,
        # vmin=-0.4, vmax=0.4,
        # symmetric_color_scale=True,
        # share_color_scale=True,
        # auto_range_cutoff=0.1,
        vmin=-0.15, vmax=0.35,
    )
    for ax in axes:
        squid.plot_polygons(ax=ax, legend=False, color='k', lw=0.75)
        field_coil.plot_polygons(ax=ax, legend=False, color='k', lw=0.75)

In [ ]:
for solution in solutions[-1:]:
    fig, axes = sc.plot_currents(
        solution,
        layers=["sample_layer", "W2", "W1"],
        figsize=(16, 6),
        streamplot=True,
        min_stream_amp=0.1,
        units="mA/um",
        cross_section_xs=None,
        cross_section_angle=0,
        share_color_scale=False,
        auto_range_cutoff=0.1,
    )
    for ax in axes:
        squid.plot_polygons(ax=ax, legend=False, color='w', lw=0.75, alpha=0.75)
        field_coil.plot_polygons(ax=ax, legend=False, color='w', lw=0.75, alpha=0.75)

## Now add a sample with $\Lambda(x, y)=10\,\mu\mathrm{m}$

In [ ]:
squid = huber_without_field_coil()

Lambda = sc.Constant(24)

layers = squid.layers
layers["sample_layer"] = sc.Layer("sample_layer", Lambda=Lambda, z0=sample_height)

films = squid.films
films["sample"] = sc.Polygon(
    "sample",
    layer="sample_layer",
    points=sample_points,
)

abstract_regions = squid.abstract_regions
abstract_regions["bounding_box"] = sc.Polygon(
    "bounding_box",
    layer=abstract_regions["bounding_box"].layer,
    points=sample_points,
)

squid.layers = layers
squid.films = films
squid.abstract_regions = abstract_regions

In [ ]:
squid.make_mesh(min_triangles=min_triangles, optimesh_steps=200)

In [ ]:
ax = squid.plot_mesh()
ax = squid.plot_polygons(ax=ax, color='k', alpha=1, lw=3, legend=False)
_ = ax.set_title(
    f"Mesh: {squid.points.shape[0]} points, "
    f"{squid.triangles.shape[0]} triangles"
)
ax.figure.set_size_inches(8,8)

In [ ]:
field_units = "mT"

fc_fields = {}
for layer in squid.layers_list:
    fc_fields[layer.z0] = field_coil_solution.field_at_position(
        squid.points,
        zs=layer.z0,
        units=field_units,
        with_units=False,
    )

def field_coil_field(x, y, z, solution=field_coil_solution, units="mT"):
    return fc_fields[z]

In [ ]:
cmap = "cividis"

fig, axes = plt.subplots(1, len(fc_fields), figsize=(15, 5))

vmin = -0.1
vmax = 0.2

vmin = -0.15
vmax = 0.35

v = np.linspace(vmin, vmax, 101)

layers = squid.layers
films = squid.films

for ax, layer in zip(axes, ["sample_layer", "W2", "W1"]):
    field = fc_fields[layers[layer].z0]
    ax.set_aspect("equal")
    ax.set_title(layer)
    im = ax.tricontourf(*squid.points.T, field, levels=v, cmap=cmap)
    ax.set_xlabel("$x$ [$\mu$m]")
    ax.set_ylabel("$y$ [$\mu$m]")
    
    squid.plot_polygons(ax=ax, lw=1, color="w", legend=False)
    
cbar = fig.colorbar(im, ax=axes, orientation="horizontal", fraction=0.1, pad=0.2)
cbar.set_ticks(np.linspace(vmin, vmax, 7))
cbar.set_label("$H_z$ / $I_\\mathrm{FC}$ [mT / mA]", fontsize=16)

In [ ]:
applied_field = field_coil_field

circulating_currents = None

solutions = sc.solve(
    device=squid,
    applied_field=applied_field,
    circulating_currents=circulating_currents,
    field_units="mT",
    current_units="uA",
    iterations=6,
    coupled=True,
)

In [ ]:
# records = []
# for s in solutions:
#     records.append(s.polygon_flux(units="Phi_0", with_units=False))
# df = pd.DataFrame.from_records(records)
# df.index.name = "Iteration"

In [ ]:
# df

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))
# ax.grid(True)
# for col in df.columns:
#     ys = df[col].values
#     ys = np.abs(np.diff(ys)[1:] / ys[1:-1])
#     xs = np.arange(len(ys)) + 1
#     ax.plot(xs, ys, 'o--', label=col)
# ax.set_ylabel("Fractional change in flux\n$(\\Phi_{(i)} -\\Phi_{(i-1)}) / \\Phi_{(i-1)}$", fontsize=14)
# ax.set_xlabel("Iteration", fontsize=14)
# ax.set_xticks(xs)
# ax.set_yscale("log")
# ax.legend(loc=0)
# fig.tight_layout()

In [ ]:
flux = solutions[-1].polygon_flux()
I_circ = squid.ureg(field_coil_solution.circulating_currents["fc_center"])
print(f"{flux['pl_hull'].to('Phi_0'):.3f~P}")
print(
    f"{(flux['pl_hull'] / I_circ).to('Phi_0/A'):.3f~P} = "
    f"{(flux['pl_hull'] / I_circ).to('pH'):.3f~P}"
)

In [ ]:
984 - 895

In [ ]:
for solution in solutions[-2:]:
    fig, axes = sc.plot_fields(
        solution,
        layers=["sample_layer", "W2", "W1"],
        units="mT",
        cmap="cividis",
        figsize=(16, 6),
        grid_shape=(400, 400),
        cross_section_xs=None,
        cross_section_angle=0,
        # vmin=-0.4, vmax=0.4,
        # symmetric_color_scale=True,
        #share_color_scale=True,
        #auto_range_cutoff=0.1,
        vmin=-0.15, vmax=0.35,
    )
    for ax in axes:
        squid.plot_polygons(ax=ax, legend=False, color='k', lw=0.75)
        field_coil.plot_polygons(ax=ax, legend=False, color='k', lw=0.75)

In [ ]:
for solution in solutions[-2:]:
    fig, axes = sc.plot_currents(
        solution,
        layers=["sample_layer", "W2", "W1"],
        figsize=(16, 6),
        streamplot=True,
        min_stream_amp=0.1,
        units="mA/um",
        cross_section_xs=None,
        cross_section_angle=0,
        share_color_scale=False,
        auto_range_cutoff=None,
    )
    for ax in axes:
        squid.plot_polygons(ax=ax, legend=False, color='w', lw=0.75, alpha=0.75)
        field_coil.plot_polygons(ax=ax, legend=False, color='w', lw=0.75, alpha=0.75)

In [ ]:
img = plt.imread("huber-image.png")
scale = np.where(img[385][:,0] == 1)[0].size / 5 # pixels per micron

origin = x0, y0 = (465, 610)

In [ ]:
polygons = {
    name: scale * sc.geometry.rotate(points, 0)
    for name, points in huber_geometry(interp_points=151).items()
}

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect("equal")
im = ax.imshow(img[:-200])
ax.set_yticks([])
ax.set_xticks([])

for name, points in polygons.items():
    if name in ["pl_hull", "fc_shield"]:
        continue
    if name in ["fc_center"]:
        points = points[20:-25]
    xs, ys = points.T
    xs = -xs
    xs = xs + x0
    ys = ys + y0
    ax.plot(xs, ys, color='w', alpha=0.8, lw=2)
    
_ = ax.set_xlim(0, img.shape[1]-1)

In [ ]:
t1 = time.time()

print(f"Elapsed time: {(t1-t0)/60:.1f} minutes")